In [9]:
import os
import sys

print("Current working directory:", os.getcwd())
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f"Added {parent_dir} to Python path")
else:
    print(f"{parent_dir} already in Python path")

Current working directory: /Users/jinceyang/Desktop/codebase/ml/learn_d2l/chap4
/Users/jinceyang/Desktop/codebase/ml/learn_d2l already in Python path


In [ ]:
import importlib
import d2l.classification as classification
from d2l.classification import FashionMNIST, SoftmaxClassifierScratch
importlib.reload(classification)

<module 'd2l.classification' from '/Users/jinceyang/Desktop/codebase/ml/learn_d2l/d2l/classification.py'>

In [11]:
import torch

In [12]:
X = torch.tensor([[1.0, 2.0, 3.0], 
                  [4.0, 5.0, 6.0]])
X.sum(dim=0, keepdim=True), X.sum(dim=1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [13]:
def softmax(X: torch.Tensor) -> torch.Tensor:
    return torch.exp(X) / torch.exp(X).sum(dim=1, keepdim=True)

In [14]:
X = torch.rand((2, 5))
X

tensor([[0.9499, 0.2532, 0.8372, 0.0189, 0.7224],
        [0.1109, 0.8826, 0.2053, 0.2591, 0.3883]])

In [15]:
X_prob = softmax(X)
X_prob, X_prob.sum(dim=1, keepdim=True)

(tensor([[0.2791, 0.1391, 0.2494, 0.1100, 0.2223],
         [0.1483, 0.3209, 0.1630, 0.1720, 0.1957]]),
 tensor([[1.0000],
         [1.0000]]))

In [17]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

In [ ]:
rng = torch.Generator().manual_seed(42)

In [ ]:
data = FashionMNIST()
model = SoftmaxClassifierScratch(
    num_features=784, 
    num_outputs=10, 
    lr=0.1, 
    rng=rng
)